<a href="https://colab.research.google.com/github/alreylz/GuitarBassClassificationCNN/blob/master/Thomann_Web_Scraping_coredamnwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup


## Conceder permiso permiso a drive para acceder al contenido de nuestra unidad.
<p> Aquí se definen algunas variables globales que se pueden usar para configurar rápidamente el siguiente conjunto de pasos o evitar que colab repita pasos como re-importar cosas. </p>


In [ ]:
#Permitimos acceso a nuestro Drive
from google.colab import drive
drive.mount("/content/drive")
MOUNT_DONE = True

COLAB_PATH = "drive/My Drive/Colab Notebooks"
DATA_PATH = "/Data/"

SCRAPING_INSTALLS_DONE = False
SCRAPING_IMPORTS_DONE = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Imports de librerías requeridas para Web Scraping
*   **requests** : Para realizar peticiones HTTP a la web objetivo, incluir parámetros en la misma y acceder a la response de forma estructurada.
*   **beautiful soup** : Para realizar el parsing de las páginas obtenidas y buscar dentro del HTML por los elementos en los que estamos interesados.


In [ ]:
#Instalación de paquetes asociados a Scraping
if SCRAPING_INSTALLS_DONE == False:
  !pip3 install selenium
  !pip3 install beautifulsoup4
  !pip3 install pandas
  !pip3 install requests
  !pip3 install lxml
  SCRAPING_INSTALLS_DONE=True
  print("[OK] Installing Scraping packages via pip")
else:
  print("[NN] Scraping packages are already installed")

if SCRAPING_IMPORTS_DONE == False:
  from selenium import webdriver
  from selenium.webdriver.common.by import By
  from selenium.webdriver.common.keys import Keys
  from selenium.webdriver.support.ui import WebDriverWait
  from selenium.webdriver.support.expected_conditions import presence_of_element_located
  from bs4 import BeautifulSoup # Para parsear encontrar y filtrar elementos en una response
  import pandas as pd
  import requests # Para hacer requests remotas
  import lxml #Librería de procesamiento de HTML y XML
  SCRAPING_IMPORTS_DONE = True
  print ("[OK] Importing Scaping tools as objects within the workspace")
else:
  print ("[NN] Scraping tools already present as objects within the workspace")


#Thomann Web Scraping Code

In [ ]:
WEBSITE_ROOT_TO_SCRAPE = "https://www.thomann.de/es/" # Aquí root website
MainSectionsToScrape = ["guitarras_y_bajos.html"] # Incluir en esta lista todas las subsecciones a explorar


testProductPage ="https://www.thomann.de/es/fender_cb_60sce_a_bass_natural_il.htm"
response = requests.get(testProductPage)


#download_and_persist_resource("https://bdbo1.thomann.de/thumb/bdb3000/pics/bdbo/13272556.jpg","example.jpg")

#for wSection in MainSectionsToScrape:
#  pageUnderAnalysis = f"{WEBSITE_ROOT_TO_SCRAPE}{wSection}"
#  print(f"{pageUnderAnalysis} is leaf : {isResultsPage(pageUnderAnalysis)}")


# PAra ver si es una página de producto hay que encontrar itemtype="http://schema.org/Product"

# Mirar estructura de página y probar crawler. DALE DUROOOO



#Las subsecciones se pueden obtener directamente comprobando si aparece un listado de elementos ()
#toScrape_Dictionary = {}
#Para cada sección que se quiere inspeccionar, accedemos a esta y creamos las subsecciones o páginas de contenidos que encontremos dentro de estas.
#for webSection in MainSectionsToScrape:
  #Accedo a la web dada por la sección y compruebo si es página de resultados
  #Si lo es contacto con esta de nuevo pero con sufijo ?ls=1000000
  # if isResultsPage(f"{WEBSITE_ROOT_TO_SCRAPE}{webSection}"):

  #Si no es página de resultados entonces guardo la lista de subsecciones y la recorro repitiendo el proceso.
  #else:
  #toScrape_Dictionary[f"{WEBSITE_ROOT_TO_SCRAPE}{webSection}"] =


#Specify here the actual webpages to go through for download
#URLS = ["https://www.thomann.de/es/cajas_signature.html"]#,"https://www.thomann.de/es/snaredrums_con_casco_de_madera.html",

#Acceso a la página
#response = requests.get(URLS[0])
#if response.status_code == 200:
#  print(f'ACCESS to {URLS[0]} WAS SUCCESSFUL')

#print(response.headers)
#print(response.text)




#soup =  BeautifulSoup(response.content,"lxml")


RESOURCE ACCESS SUCCESSFUL


**Funciones Auxiliares**



In [ ]:
#Funciones de identificación de errores / Tipos de páginas
# isThomannsResultsPage: dado un objeto Beautiful Soup, comprueba si este contiene lo que caracteriza a una página de resultados
def isThomannsResultsPage(bSoupPage):
    if(len(bSoupPage.findAll('div',attrs={'class':'results'})) == 0):
      return False
    return True
# isErrorPage: Dado un objeto response, indica si esta es una página de error
def isErrorPage(response):
  if(response.ok):
    return False
  return True
# isSubcategoriesPage:  devuelve una lista de subcategorías si la página proporcionada (ya parseada - Beautiful Soup- ) es una página de categorías
def isSubcategoriesPage(bsObject):
  subCatsDiv = bsObject.findAll("div",attrs={'class':'subcategories'})
  if(subCatsDiv is not None and len(subCatsDiv)>0):
    return subCatsDiv
  return None
# isProductListPage:  devuelve True si la url como input es una página de resultados
def isProductListPage(wUrl, outBS=[]):
  listProdListings = BeautifulSoup(requests.get(wUrl).content,'html.parser').findAll('div',attrs={'class':'rs-productlisting'})
  if(len(listProdListings) > 0 ):
    for pl in listProdListings:
      outBS.append(pl)
    return True
  return False

#Funciones de filtro:
#A. Filtro para página de categoría de más alto nivel (selección de instrumentos)
def subsectGuitarrasYBajos(name):
  if(name.startswith("Guitarras") or name.startswith("Bajos")):
    return True
  return False
#B. Obtención de páginas de resultados de Guitarras eléctricas:
def subsectNameModelsyGuitarras(name):
  if(name.startswith("Modelos") or name.startswith("Guitarra")):
    return True
  return False
#C. Filtro detallado para subcategorías.
def filterClean(name):
  if(name.startswith("Sets") or name.startswith("Otros") or name.startswith("Masterclass") or name.startswith("Tamaños")):
    return False
  else:
    return True



# Funciones para extraer datos de todos los elementos de una página de resultados:
<ul>
<li>
<p><b>getIndivItemInfoFromSearchListResponse</b> </p> Extrae detalles de producto de una response de página de resultados (e.g. actúa sobre una página solamente)
</li>
<li>
<p><b>GetAllProductsSummaryFromSearchPage</b> </p> Extrae los datos de productos de todas las páginas de resultados asociadas, iterando en estas.
</li>
</ul>

![ejemplo página de resultados](https://image.prntscr.com/image/IM288yDVQ-yVNCU4T6Hqqg.png)


In [ ]:
#Extracción de datos de la página contenida en objeto response
def getIndivItemInfoFromSearchListResponse(responseObj, debug=False):

  bweb = BeautifulSoup(responseObj.content, 'html.parser')
  if(not isThomannsResultsPage(bweb)):
    print("Input url is not a results Page.")
    return []

  #1.Selección de todos los contenedores de datos sobre artículos; elementos con clase ('extensible-article')
  prodContainerList = bweb.select("div.articles-list > div.extensible-article")
  #print(prodContainerList[0].prettify())

  #2. Detalles tanto en parte izqda como dcha
  listaImsProds = bweb.select("div.left div.product-image.image-block")
  listaDetallesProds = bweb.select("div.right .top")

  if(debug):
    print(f"Productos encontrados {len(listaDetallesProds)}")
  listProdsInfoInPage = [] # Lista de output

  pIt = 0
  #LISTA DE PRODUCTOS CON SUS CARACTERÍSTICAS PRINCIPALES:
  for elem in listaDetallesProds:

    urlProd = listaImsProds[pIt].select_one("a.article-link.link")
    urlProd = 'N/A' if urlProd is None else urlProd['href']
    if(debug):
      print(f"url: {urlProd}")

    thumbnail = listaImsProds[pIt].select_one("a.article-link.link img.image")
    thumbnail = 'N/A' if thumbnail is None else thumbnail['src']
    if(debug):
      print(f"thumbnail: {thumbnail}")


    manuf = elem.select_one('.head span.manufacturer')
    manuf = 'N/A' if manuf is None else manuf.text
    if(debug):
      print(f"Manufacturer: {manuf}")

    name = elem.select_one(".head .model")
    name =  'N/A' if (name is None) else name.text
    if(debug):
      print(f"Name: {name}")

    subcaption = elem.select_one('div.subtitle-block div')
    subcaption = 'N/A' if (subcaption is None) else subcaption.text
    if(debug):
      print(f"Subcaption: {subcaption} ")

    price = elem.select_one('div.body .price-block span.article-basketlink')
    price = 'N/A' if (price is None) else price.text
    if(debug):
      print(f"Price: {price}")

    listProdsInfoInPage.append((urlProd,thumbnail,manuf,name,subcaption,price))

    pIt+=1

    if(debug):
      print("\n -------------------")

  return listProdsInfoInPage

# GetProductSummaryFromSearchPage :
#   Input
#     resultsPageURl: url de una página que sea de resultados
#
#   Return:
#       null O lista de tuplas con los datos de todos los productos encontrados
def GetAllProductsSummaryFromSearchPage(resultsPageUrl ,pagination=True, debug=True):

  listAllProdsInfo = []

  #1. Realizamos request para obtener la página HTML
  getMethodParams = { 'ls': '100'}
  response = requests.get(resultsPageUrl, params=getMethodParams)

  #(*) Error checking
  if(isErrorPage(response)):
    print("Error Page encountered.")
    return listAllProdsInfo

  bweb = BeautifulSoup(response.content, 'html.parser')

  if(not isThomannsResultsPage(bweb)):
    print("Input url is not a results Page.")
    return listAllProdsInfo

  print(f"[GetProductSummaryFromSearchpage()] Exploring Results for: {bweb.find('h1').text} \n **************************")

  numPagesToExplore = 1
  #2. Si queremos iterar sobre las páginas de resultados, entonces hay que determinar cuántas hay
  if(pagination == True):
    if debug:
      print(f"Getting number of result pages :  {response.url}")
    # Encontramos el número de páginas (para eso podemos mirar el último elemento del DOM que hay con la el atributo class="page")
    list_page_tags = bweb.find_all("div",attrs={'class':'page'})
    # Obtenemos el número de página máximo (el último elemento de los tags que satisfacen class="page")
    numPagesToExplore = numPagesToExplore =  int(list_page_tags[-1].a.text) if (len(list_page_tags)>0) else 1

  if debug:
    print(f"Going through a total of : {numPagesToExplore} pages \n ........................")

  #3. Hago requests a cada página de resultados y obtengo las urls de productos.
  for pg in range(numPagesToExplore):
    pg+=1
    getMethodParams2 = { 'ls':'100', 'pg': f'{pg}' }
    rp = requests.get(resultsPageUrl, params=getMethodParams2)
    rp_Soup = BeautifulSoup(rp.content, 'html.parser')
    if debug:
      print(f"...Page {pg} -> {response.url}?pg={pg}")

    listAllProdsInfo.append(getIndivItemInfoFromSearchListResponse(rp, debug=False))

  return listAllProdsInfo

#image_list = BeautifulSoup(response.content,"lxml").find_all(isProduct)
#    for image in image_list :
#      imgLink = image.find('img').get('src')
#      print(f"Downloading image from: {imgLink}")
#      download_and_persist_resource(imgLink, fileName=str(image_count))
#      image_count+=1

In [ ]:
#@title [Tests] **Result Page Data Extraction**
testURL = "https://www.thomann.de/es/modelos_st.html?ls=100"#@param {type:"string"}
responseI = requests.get(testURL)

#Obtenemos una lista de tuples de datos de todos los elementos en una página dada.
listDataFromPage = getIndivItemInfoFromSearchListResponse(responseI,debug=False)

print(f"Total de productos en {testURL} = {len(listDataFromPage)}")
print(f"Datos del producto #1 de la misma:  (urlProducto, urlImagen, Marca, Modelo, Detalle , Precio")
print(f"\t {listDataFromPage[0]}")
print("\n\n")


#Pruebas recorriendo las páginas de elementos: se obtiene una lista de tuplas por página.
listAll = GetAllProductsSummaryFromSearchPage("https://www.thomann.de/es/modelos_st.html",debug=False)

print(f"Total de páginas exploradas {len(listAll)}")
print(f"Se obtiene una lista de tuplas de Producto por cada página explorada:")
print(f"listAll[0] es lista de tuplas de productos de la primera página de {len(listAll[0])} resultados: \n\t {listAll[0]}")



## Obtención de categorías que contienen guitarras y bajos:

In [ ]:
# BUGGY!
# getInterestCategoriesFromSubsectionPage: Obtiene los enlaces asociados a las subsecciones que queramos, cuyo nombre está filtrado por la función filterFCategoryName
# Input:
#   categoriesPageUrl: string que contiene la página web a la que acceder y que ha de ser una página de categorías para que funcione correctamente.
#   filterFCategoryName: función que devuelve True or False, con un parámetro (subsectionName) que permite definir las cadenas que queremos filtrar
#   isRecursive: permite una búsqueda recursiva para solamente obtener al final páginas de resultados, nunca otras páginas de categorías.
#   debug: si es True, muestra todos los side effects de esta función.
# Output:
#   return ->  lista de tuplas (path en la página, urlAResultados)
def getInterestCategoriesFromSubsectionPage( categoriesPageUrl ,filterFCategoryName = None, isRecursive=False, debug=True, breadcrumb="", recList=[]):

  if (debug):
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print(f"[getInterestCategoriesFromSubsectionPage()]: Traversing {categoriesPageUrl}")
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
  responseCategoriesPage =  requests.get(categoriesPageUrl)
  pageSoup = BeautifulSoup(responseCategoriesPage.content, "html.parser")

  listElems =  isSubcategoriesPage(pageSoup)
  if(listElems is None):
    print(f"Input {categoriesPageUrl} is not a subcategories Page")
    return categoriesPageUrl

  #Obtengo lista de links a subsecciones
  anchorList = listElems[0].findAll('a',attrs={'class':'link'})
  #Compruebo nombre de las subsecciones
  if (debug):
    print("Subsecciones encontradas:\n -----------------")
  aIndex = 0 # Uso auxiliar para filtro de todos las las subsecciones
  interestAnchorList = recList # Lista para guardar los enlaces a subsecciones de interés.
  for a in anchorList:
      if (debug):
        print(f" - {a.find('span').text}")
      subsectionName = a.find('span',attrs={'class':'name'}).text.strip()
      if(filterFCategoryName is not None ):
        if(filterFCategoryName(subsectionName)):
          savedUrl = anchorList[aIndex]['href']
          #Caso recursivo: Si la página a guardar es una de subcategorías
          if( (not isProductListPage(savedUrl)) and isRecursive == True):
            # Llamada a esta función de nuevo
            print("*LLAMADA RECURSIVA")
            getInterestCategoriesFromSubsectionPage(savedUrl,filterFCategoryName,isRecursive=True, breadcrumb=f"{breadcrumb+subsectionName} > ", recList=interestAnchorList)
            return
          #Si la página obtenida a guardar es una página de resultados, ya hemos obtenido lo que queríamos.
          elif(isProductListPage(savedUrl) and isRecursive == True):
            interestAnchorList.append( ((breadcrumb+subsectionName),savedUrl))
          else:
            interestAnchorList.append( ((breadcrumb+subsectionName),savedUrl ))
      #Sin filtro y sin recursiveness
      else:
        interestAnchorList.append((breadcrumb+subsectionName,anchorList[aIndex]['href']))
      aIndex+=1
  print("\n**********************")
  if (debug):
    #Resultado de filtro de subsecciones:
    print("Subsecciones extraídas:\n -----------------")
    for ss in interestAnchorList:
      print(ss)
  if (debug):
    print("\n\n")
  return interestAnchorList


In [ ]:
#Obtención de lista de categorías estrictamente asociadas a guitarras y bajos (usamos función filtro subsectGuitarrasYBajos)
print("[TEST 1]")
print("Listado de categorías en primer nivel de búsqueda (no recursiva):")
GuitarAndBassSubSects = getInterestCategoriesFromSubsectionPage("https://www.thomann.de/es/guitarras_y_bajos.html", subsectGuitarrasYBajos, isRecursive=False, debug=False)
print("Cada elemento de la lista es tupla : (NombreDeSubsección, urlASubsecciónOPáginaDeResultados)")
print(f"\t GuitarAndBassSubSects[0] = {GuitarAndBassSubSects[0]}")
print("[TE 1]******************")

print("[TEST 2]")
urlTestRecursive = "https://www.thomann.de/es/guitarras_y_bajos.html"
print("Búsqueda recursiva de categorías que pueden ser de interés hasta llegar a una página de resultados, que son las que analizaremos para crear el dataset: ")
#Exploración recursiva de subsecciones usando un filtro (subsectGuitarrasYBajos), para obtener lista de páginas de resultados.
aList = getInterestCategoriesFromSubsectionPage(urlTestRecursive, subsectGuitarrasYBajos ,isRecursive=True, debug=False)
print("Resultado final: ")
for l in aList:
  print(l)
print("[TE 2]******************")

In [ ]:
responseCategoriesPage =  requests.get("https://www.thomann.de/es/guitarras_y_bajos.html")
pageSoup = BeautifulSoup(responseCategoriesPage.content, "html.parser")
listElems =  pageSoup.findAll("div",attrs={'class':'subcategories'})

print(len(listElems))
print(f"LISTA: {listElems}")
#Obtengo lista de links a subsecciones
anchorList = listElems[0].findAll('a',attrs={'class':'link'})
#Compruebo nombre de las subsecciones
print("Subsecciones encontradas:\n -----------------")
aIndex = 0 # Uso auxiliar para filtro de todos las las subsecciones
interestAnchorList = [] # Lista para guardar los enlaces a subsecciones de interés.
for a in anchorList:

    print(f" - {a.find('span').text}")
    subsectionName = a.find('span',attrs={'class':'name'}).text.strip()
    if (subsectionName.startswith("Guitarras") or subsectionName.startswith("Bajos")):
      interestAnchorList.append((subsectionName,anchorList[aIndex]['href']))

    aIndex+=1

print("\n\n")

#Resultado de filtro de subsecciones
print("Subsecciones de interés (lista de duplas):\n -----------------")
for ss in interestAnchorList:
  print(ss)

In [ ]:



bsIfResutlsPage = []

#for ss in interestAnchorList:
#ssurl = interestAnchorList[2][-1]#url que asumimos que es de lista de productos pero nope
ssurl = "https://www.thomann.de/es/modelos_st.html"

#permite comprobar si una página es de resultados y recoger los datos dentreo del product listing.
#print(isProductListPage(ssurl,bsIfResutlsPage))


#Obtener urls de cada elemento de una página de resultados
def extractProdPageUrlsFromProductListPage(resultsPageUrl,debug=True):
  listProdUrls = []

  image_count = 0
  getMethodParams = { 'ls': '100'}
  response = requests.get(resultsPageUrl, params=getMethodParams)
  if(isErrorPage(response)):
    return null

  bweb = BeautifulSoup(response.content, 'lxml')
  print(f"Exploring Results for: {bweb.find('h1').text}\n **************************")

  if debug:
    print(f"[STEP 1] - Getting number of result pages :  {response.url}")

  # Encontrar número de páginas (para eso podemos mirar el último elemento del DOM que hay con la el atributo class="page")
  list_page_tags = bweb.find_all("div",attrs={'class':'page'})
  # Obtenemos el número de página máximo (el último elemento de los tags que satisfacen class="page")

  numPagesToExplore =  int(list_page_tags[-1].a.text) if (len(list_page_tags)>0) else 1

  if debug:
    print(f"[STEP 2] - Get actual urls go through from total of : {numPagesToExplore}")
  # Hago requests a cada página de resultados y obtengo las urls de productos.
  for pg in range(numPagesToExplore):
    pg+=1
    getMethodParams2 = { 'ls':'100', 'pg': f'{pg}' }
    response_Pg = BeautifulSoup(requests.get(resultsPageUrl, params=getMethodParams2).content, 'html.parser')

    if debug:
      print(f"...Page {pg} -> {response.url}?pg={pg}")

    #Div-product image
    bsLiContainer = response_Pg.find('div',attrs={'class':'articles-list'}).findAll('div',class_='product-image')
    print(f"Num results : {len(bsLiContainer)}")

    #print(len(listProdAnchors))
    for a in bsLiContainer:
      listProdUrls.append(a.find('a',attrs={'class':'article-link'},href=True)['href'])


  return listProdUrls


myURLs = extractProdUrlsFromProductListPage("https://www.thomann.de/es/modelos_st.html",True)

print(f"Total links obtained (total number of products): {len(myURLs)}")










#print(bsIfResutlsPage[0].prettify())

#soupAgain = BeautifulSoup(requests.get(ssurl, params={'ls':'100'}).content)





#lProduct = soupAgain.findAll('div',attrs={'class':'rs-productlisting'})
#print(len(lProduct)>0)

NameError: ignored

## Implementation: Descarga de Imágenes

In [ ]:
# Descargar un archivo dada una URL al recurso
def download_and_persist_resource(url, fileName ,destDir="/PA-Dataset/", basePath="drive/My Drive/Colab Notebooks"):
  print(f"Copying {url} file to Colab")
  if fileName == None: # Nombre por defecto es el tiene el recurso en la web original; cogemos la url, y extraemos lo que pone tras el último '/'nombreDeArchivo
    fileName = url.split('/')[-1]
  fullPath =f"{basePath}{destDir}{fileName}"
  print(f"Saving resource to: {fullPath}")
  nu_file = open(f"{fullPath}",'wb')
  nu_file.write(requests.get(url).content)
  nu_file.close()


## Main Loop
Partiendo de la página de Guitarras/Bajos de Thomann:
1. Obtenemos todas las páginas de resultados en aquellas subsecciones que son precisamente Guitarras o Bajos. El resto de instrumentos no directamente relacionados (e.g. Ukeleles) son descartados: las categorías que nos quedarán serán las que se ven a continuación:
![Categorías a explorar](https://image.prntscr.com/image/gVY1xIqgR1OvkRrceoa-HQ.png)
<br>Esto lo guardamos en un diccionario con la tipología de guitarras o bajos de más alto nivel.

2.Accedemos a todas las urls provistas y realizamos un escaneo de todas las páginas de resultados derivadas, de forma que cubramos todos los resultados de la web en cuanto a Guitarras y Bajos


In [ ]:
#Dictionary from top level categorization
AllSubsects = {}

baseUrlCategory = "https://www.thomann.de/es/guitarras_y_bajos.html"

#Obtenemos las los tipos de nivel 1 y de ellos obtenemos un diccionario
GuitarsAndBassTypesListUrls = getInterestCategoriesFromSubsectionPage(baseUrlCategory, subsectGuitarrasYBajos, isRecursive=False, debug= False )
print(f"1. Lista de tuplas : \n {GuitarsAndBassTypesListUrls}")
print("***********************************")

print("La recursividad va como el culo! La has liado")

print(f"2. Lista de tuplas : \n {GuitarsAndBassTypesListUrls}")






#Atravesamos las categorías existentes en todos los elementos
#for sub in GuitarsAndBassTypesListUrls:
#  print(f"Explorando Sub-Categoría [{sub[0]}][{sub[1]}]. Buscando páginas de resultados asociadas: \n*********************")
#  AllSubsects[f'{sub[0]}'] = getInterestCategoriesFromSubsectionPage(sub[1],filterClean, isRecursive=True, breadcrumb=f"{sub[0]} > ",debug=False)

#print(f"Listado de páginas de resultados para cada elemento (Dictionary Key) ,[(Subcat , url)]:")
#for su in AllSubsects:
#  print(f"{su} : {AllSubsects[su]}")

#electricGuitarVarieties =getInterestCategoriesFromSubsectionPage("https://www.thomann.de/es/guitarras_electricas.html", subsectName, debug=True)


**********************
1. Lista de tuplas : 
 [('Guitarras eléctricas', 'https://www.thomann.de/es/guitarras_electricas.html'), ('Guitarras clásicas', 'https://www.thomann.de/es/guitarras_clasicas.html'), ('Guitarras acústicas', 'https://www.thomann.de/es/guitarras_acusticas.html'), ('Bajos eléctricos', 'https://www.thomann.de/es/bajos_electricos.html'), ('Bajos acústicos / semiacústicos', 'https://www.thomann.de/es/bajos_acusticos_semiacusticos.html'), ('Guitarras eléctricas > Modelos ST', 'https://www.thomann.de/es/modelos_st.html'), ('Guitarras eléctricas > Modelos T', 'https://www.thomann.de/es/modelos_t.html'), ('Guitarras eléctricas > Modelos Single Cut', 'https://www.thomann.de/es/modelos_lp.html'), ('Guitarras eléctricas > Modelos Double Cut', 'https://www.thomann.de/es/modelos_sg.html'), ('Guitarras eléctricas > Guitarra Hollowbody', 'https://www.thomann.de/es/guitarra_hollowbody.html'), ('Guitarras eléctricas > Guitarras Heavy', 'https://www.thomann.de/es/guitarras_heavy.htm

In [ ]:
# 2 Para cada tupla, extraigo los datos de cada producto: obtengo una lista de [(subcategoría, [Lista de tuplas de productos])]
prodTypeAndListProds = {}
for resPage in GuitarsAndBassTypesListUrls:
  if(">" in resPage[0]):

    prodType = 'Guitarra' if ("Guitarra" in resPage[0]) else "Bajo"
    if(not(f"{prodType}" in prodTypeAndListProds)):
      prodTypeAndListProds[f"{prodType}"] = GetAllProductsSummaryFromSearchPage(resPage[1], debug=False)
    else:
      prodTypeAndListProds[f"{prodType}"].extend(GetAllProductsSummaryFromSearchPage(resPage[1], debug=False))





[GetProductSummaryFromSearchpage()] Exploring Results for:     Modelos ST    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Modelos T    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Modelos Single Cut    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Modelos Double Cut    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Guitarra Hollowbody    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Guitarras Heavy    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Guitarras para zurdos    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Guitarras de gama alta    
 **************************
[GetProductSummaryFromSearchpage()] Exploring Results for:     Guitarras Jazz    
 *****************

In [ ]:
print("Diccionario resultante:")
print(f"Número de keys = {len(prodTypeAndListProds)} : {prodTypeAndListProds.keys()}")
print(f"Páginas de resultados exploradas (Bajo): {len(prodTypeAndListProds['Bajo'])}")
print(f"Páginas de resultados exploradas (Guitarra): {len(prodTypeAndListProds['Guitarra'])}")








#print(f" Acceso a resultados de una sola página de guitarras: {prodTypeAndListProds['Guitarra'][0]}")
# recorro los productos para la descarga de las imágenes:
#for resInnerPage in prodTypeAndListProds:




Diccionario resultante:
Número de keys = 2 : dict_keys(['Guitarra', 'Bajo'])
Páginas de resultados exploradas (Bajo): 29
Páginas de resultados exploradas (Guitarra): 95
('https://www.thomann.de/es/fender_sq_cv_50_strat_mn_sp_ed._fr.htm', 'https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/490634.jpg', 'Fender', 'SQ CV 50 Strat MN Sp.Ed. FR', 'Guitarra eléctrica Stratocaster', '435 €')


Finally Downloading OMG:


In [ ]:
#Acceder a imágenes de guitarras y guardarlas

imgId = 0
for pgnIt in prodTypeAndListProds['Guitarra']:
  for itemIt in pgnIt:
    if(not "sorrynopic" in itemIt[1]):
      #print(f"{itemIt[1]}")
      imgId+=1
      download_and_persist_resource(itemIt[1],f"Guitar{str(imgId)}.jpg",destDir="/PA-Dataset/Guitarras/thumbnail/", basePath="drive/My Drive/Colab Notebooks")


print(f"{imgId}")


In [ ]:
#Acceder a imágenes de bajos y guardarlas


imgId = 0
for pgnIt in prodTypeAndListProds['Bajo']:
  for itemIt in pgnIt:
    if(not "sorrynopic" in itemIt[1]):
      #print(f"{itemIt[1]}")
      imgId+=1
      download_and_persist_resource(itemIt[1],f"Bass{str(imgId)}.jpg",destDir="/PA-Dataset/Bajos/thumbnail/", basePath="drive/My Drive/Colab Notebooks")
      time.sleep(0.1)


print(f" Finished: {imgId} imágenes de Bajos guardadas con éxito")


Copying https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/446264.jpg file to Colab
Saving resource to: drive/My Drive/Colab Notebooks/PA-Dataset/Bajos/thumbnail/Bass1.jpg
Copying https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/304930.jpg file to Colab
Saving resource to: drive/My Drive/Colab Notebooks/PA-Dataset/Bajos/thumbnail/Bass2.jpg
Copying https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/351690.jpg file to Colab
Saving resource to: drive/My Drive/Colab Notebooks/PA-Dataset/Bajos/thumbnail/Bass3.jpg
Copying https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/446377.jpg file to Colab
Saving resource to: drive/My Drive/Colab Notebooks/PA-Dataset/Bajos/thumbnail/Bass4.jpg
Copying https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/446376.jpg file to Colab
Saving resource to: drive/My Drive/Colab Notebooks/PA-Dataset/Bajos/thumbnail/Bass5.jpg
Copying https://thumbs.static-thomann.de/thumb/thumb150x150/pics/prod/445968.jpg file

In [ ]:

time.sleep(0.1)
print("HI")
time.sleep(1)
print("HO")

HI
HO


<h3>Celda de test (no loop): Obtención de información de todos los artículos de una página de resultados:</h3>
<p> Usando selectores de CSS se hace mucho más sencillo (e.g. .select() .select_one() <p/>


In [ ]:
urlTest = "https://www.thomann.de/es/modelos_st.html"

getMethodParams = { 'ls': '100'}
response = requests.get(urlTest, params=getMethodParams)

if(isErrorPage(response)):
  print("Error Page encountered.")
bweb = BeautifulSoup(response.content, 'lxml')
if(len(bweb.findAll("div",attrs={'class':'results'})) == 0):
  print("Input url is not a results Page.")

#Selección de todos los contenedores de datos sobre artículos ('extensible-article)
prodContainerList = bweb.select("div.articles-list > div.extensible-article")
#print(prodContainerList[0].prettify())

#Detalles tanto en parte izqda como dcha
listaDetallesProds = bweb.select("div.left div.product-image.image-block , div.right .top")

listProdsInfoInPage  = []

#LISTA DE PRODUCTOS CON SUS CARACTERÍSTICAS PRINCIPALES:
for elem in listaDetallesProds:

  urlProd = elem.select_one("a.article-link.link")
  urlProd = 'N/A' if urlProd is None else urlProd['href']
  print(f"url: {urlProd}")

  thumbnail = elem.select_one("a.article-link.link img.image")
  thumbnail = 'N/A' if thumbnail is None else thumbnail['src']
  print(f"thumbnail: {thumbnail}")

  manuf = elem.select_one('.head span.manufacturer')
  manuf = 'N/A' if manuf is None else manuf.text
  print(f"Manufacturer: {manuf}")

  name = elem.select_one(".head .model")
  name =  'N/A' if (name is None) else name.text
  print(f"Name: {name}")

  subcaption = elem.select_one('div.subtitle-block div')
  subcaption = 'N/A' if (subcaption is None) else subcaption.text
  print(f"Subcaption: {subcaption} ")

  price = elem.select_one('div.body .price-block span.article-basketlink')
  price = 'N/A' if (price is None) else price.text
  print(f"Price: {price}")

  listProdsInfoInPage.append((urlProd,thumbnail,manuf,name,subcaption,price))

  print("\n -------------------")

#LISTA DE TUPLES con toda la información más relevante de un artículo
for it in listProdsInfoInPage:
  print(it)




In [ ]:
#Devuelve un boolean que define si el tag de HTML 'elem' contiene una clase dada por el argumento 'className'
def IsMatchingClass(elem, className="page"):
  listClasses = elem.get_attribute_list('class')
  for class_ in listClasses:
    if class_ == className:
      return True
  return False
#Función de filtro obtener exclusivamente enlaces de productos
def isProduct(tagElem):
  class_list = tagElem.get_attribute_list('class')
  for cl in class_list:
    if cl == "article-link" and tagElem.find('img') is not None:
        return True
  return False

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('raccoon_labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()

In [ ]:
from PIL import Image
import numpy as np
import math
from google.colab import files

crop_path = "drive/My Drive/Colab Notebooks/Data/ganSrc/"

print("Obtaining list of files!")
files_in = !ls '$crop_path'
print(files_in[:' '])
#Obtengo array de strings donde estan todos los numeros pero en varias lineas
files_in[:] = ''.join(files_in[:])
print("Making each one elem of list!")
print(files_in)
#for l in files_in:


print(dirs)
#fileStream = open(f"{crop_path}",'r')
#print(fileStream)

image = Image.open(f"{crop_path}0")
#Get image dimensions
width, height = image.size

#Get closest power of 2 in order to crop appropiately
nu_width = math.pow(2,int(math.log(width,2)))
nu_height = math.pow(2,int(math.log(height,2)))

print(f"Cropping image: ")
print(f" WIDTH: {width}-->{nu_width}")
print(f" HEIGHT: {height}-->{nu_height}")
#crop((left, top, right, bottom))
cropped_img = image.crop((0,0,nu_width,nu_height))

print(cropped_img)

cropped_img.save(fp=f"{crop_path}0.jpg")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread(f"{crop_path}/0.jpg")
imgplot = plt.imshow(img)
plt.show()